In [1]:
%cd /Users/gokhankocmarli/Projects/bitirme_tezi/div2k_realesrgan_hat_trainer/

/Users/gokhankocmarli/Projects/bitirme_tezi/div2k_realesrgan_hat_trainer


# Preperation of the Environment & Dataset

**Dowload the original HAT codebase.**

In [1]:
!git clone https://github.com/XPixelGroup/HAT.git .codebases/HAT

/Users/gokhankocmarli/Projects/bitirme_tezi/div2k_realesrgan_hat_trainer


**Install dependencies.**

In [ ]:
%pip install torch torchvision torchaudio
%pip install -r .codebases/HAT/requirements.txt
!python .codebases/HAT/setup.py develop

**Download dataset DIV2K and unzip it.**

In [ ]:
!wget -c http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_HR.zip -P .datasets/DIV2K
!unzip .datasets/DIV2K/DIV2K_train_HR.zip -d .datasets/DIV2K
!wget -c http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_valid_HR.zip -P .datasets/DIV2K
!unzip .datasets/DIV2K/DIV2K_valid_HR.zip -d .datasets/DIV2K
!wget -c http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_LR_bicubic_X2.zip -P .datasets/DIV2K
!unzip .datasets/DIV2K/DIV2K_train_LR_bicubic_X2.zip -d .datasets/DIV2K
!wget -c http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_valid_LR_bicubic_X2.zip -P .datasets/DIV2K
!unzip .datasets/DIV2K/DIV2K_valid_LR_bicubic_X2.zip -d .datasets/DIV2K


**Rename the directories.**

In [ ]:
!mv .datasets/DIV2K/DIV2K_train_HR .datasets/DIV2K/train_hr
!mv .datasets/DIV2K/DIV2K_valid_HR .datasets/DIV2K/valid_hr
!mv .datasets/DIV2K/DIV2K_train_LR_bicubic/X2 .datasets/DIV2K/train_lr
!mv .datasets/DIV2K/DIV2K_valid_LR_bicubic/X2 .datasets/DIV2K/valid_lr
!rm -rf .datasets/DIV2K/DIV2K_train_LR_bicubic
!rm -rf .datasets/DIV2K/DIV2K_valid_LR_bicubic

**Get the mutliscale converter & sub-image maker scripts from Real-ESRGAN codebase.**

In [ ]:
!mkdir .scripts
!wget https://raw.githubusercontent.com/xinntao/Real-ESRGAN/master/scripts/generate_multiscale_DF2K.py -P .scripts
!wget https://raw.githubusercontent.com/xinntao/Real-ESRGAN/master/scripts/extract_subimages.py -P .scripts
!wget https://raw.githubusercontent.com/xinntao/Real-ESRGAN/master/scripts/generate_meta_info.py -P .scripts

**Generate multiscale images.**

In [ ]:
!python .scripts/generate_multiscale_DF2K.py --input .datasets/DIV2K/train_hr --output .datasets/DIV2K/train_hr_multiscale
!python .scripts/generate_multiscale_DF2K.py --input .datasets/DIV2K/valid_hr --output .datasets/DIV2K/valid_hr_multiscale
!python .scripts/generate_multiscale_DF2K.py --input .datasets/DIV2K/train_lr --output .datasets/DIV2K/train_lr_multiscale
!python .scripts/generate_multiscale_DF2K.py --input .datasets/DIV2K/valid_lr --output .datasets/DIV2K/valid_lr_multiscale

**Generate sub-images.**

In [ ]:
!python .scripts/extract_subimages.py --input .datasets/DIV2K/train_hr --output .datasets/DIV2K/sub_train_hr --crop_size 400 --step 200
!python .scripts/extract_subimages.py --input .datasets/DIV2K/valid_hr --output .datasets/DIV2K/sub_valid_hr --crop_size 400 --step 200
!python .scripts/extract_subimages.py --input .datasets/DIV2K/train_lr --output .datasets/DIV2K/sub_train_lr --crop_size 200 --step 100
!python .scripts/extract_subimages.py --input .datasets/DIV2K/valid_lr --output .datasets/DIV2K/sub_valid_lr --crop_size 200 --step 100

In [ ]:
# Remove the original, and mutliscale images.
!rm -r .datasets/DIV2K/train_hr
!rm -r .datasets/DIV2K/valid_hr
!rm -r .datasets/DIV2K/train_lr
!rm -r .datasets/DIV2K/valid_lr
!rm -r .datasets/DIV2K/train_hr_multiscale
!rm -r .datasets/DIV2K/valid_hr_multiscale
!rm -r .datasets/DIV2K/train_lr_multiscale
!rm -r .datasets/DIV2K/valid_lr_multiscale

**Check the size of the dataset.**

In [30]:
import os

# Check the sizes of the datasets' image count.
EQUALITY_CHECK = {
    "sub_train_hr": "sub_train_lr",
    "sub_valid_hr": "sub_valid_lr",
}

for hr, lr in EQUALITY_CHECK.items():
    hr_path = os.path.join(".datasets/DIV2K", hr)
    lr_path = os.path.join(".datasets/DIV2K", lr)
    hr_images = os.listdir(hr_path)
    lr_images = os.listdir(lr_path)
    if len(hr_images) != len(lr_images):
        print(f"[ERROR] Number of images in {hr} and {lr} are not equal.")
    print(f"[<OK>] Number of images in {hr} and {lr} are equal.")


# Check the total disk usage.
!echo "\nTotal Disk Usage"
!du -sh .datasets/DIV2K/

[<OK>] Number of images in sub_train_hr and sub_train_lr are equal.
[<OK>] Number of images in sub_valid_hr and sub_valid_lr are equal.

Total Disk Usage
 22G	.datasets/DIV2K/


# Select a Subset of Whole Dataset for Faster Training

In [31]:
from glob import glob
from os.path import join, relpath
from cv2 import imread
from numpy.random import rand

# Change these variables to your own dataset location.
METADATAS = [
    {
        'root_location': '.datasets/DIV2K/',
        'dataset_location': '.datasets/DIV2K/sub_train_hr/',
        'meta_data_location': '.datasets/DIV2K/meta_info_train.txt'
    },
    {
        'root_location': '.datasets/DIV2K/',
        'dataset_location': '.datasets/DIV2K/sub_valid_hr/',
        'meta_data_location': '.datasets/DIV2K/meta_info_valid.txt'
    }
]

# Select 20% of each dataset for faster training, and place them into metadata.
for meta_data in METADATAS:
    with open(meta_data['meta_data_location'], 'w') as meta_info_file:
        # Get all image paths.
        all_files = sorted(glob(join(meta_data['dataset_location'], '*')))
        
        # Traverse all images.
        for image_path in all_files:

            # Select a random number, if it is greater than 0.2, skip this image.
            if rand() > 0.2:
                continue

            # Check the image.
            try:
                image = imread(image_path)
            except (IOError, OSError) as error:
                print(f'Read {image_path} error: {error}')
                continue
            if image is None:
                print(f'Img is None: {image_path}')
                continue

            # Get the relative path.
            img_name = relpath(image_path, meta_data['root_location'])
            
            # Write into metadata.
            meta_info_file.write(f'{img_name}\n')

# Generate Configuration File for Training

**Copy a new configuration file from the original one.**

In [41]:
!mkdir -p .options/train
!cp .codebases/HAT/options/train/train_HAT-S_SRx2_from_scratch.yml .options/train/hat_2x_div2k.yml

In [42]:
# Read the configuration YAML file and change the parameters needed.
import yaml

with open('.options/train/hat_2x_div2k.yml', 'r') as file:
    config = yaml.safe_load(file)

if config['datasets']['train']['name'] != "DIV2K_Training":
    config['datasets']['train']['name'] = "DIV2K_Training"
    config['datasets']['train']['dataroot_gt'] = ".datasets/DIV2K/"
    config['datasets']['train']['dataroot_lq'] = ".datasets/DIV2K/sub_train_lr" # It needs sub-dir as well.
    config['datasets']['train']['meta_info_file'] = ".datasets/DIV2K/meta_info_train.txt"
    config['datasets']['val']['name'] = "DIV2K_Validation"
    config['datasets']['val']['dataroot_gt'] = ".datasets/DIV2K/"
    config['datasets']['val']['dataroot_lq'] = ".datasets/DIV2K/sub_valid_lr"  # It needs sub-dir as well.
    del config['datasets']['val_2']
    del config['datasets']['val_3']

    # Save the configuration YAML file.
    with open('.options/train/hat_2x_div2k_test.yml', 'w') as file:
        yaml.dump(config, file, sort_keys=False)

# Demo Training (Short Period & w/ Debugging)

In [37]:
!python .codebases/HAT/hat/train.py -opt .options/train/hat_2x_div2k.yml --debug

/Users/gokhankocmarli/Projects/bitirme_tezi/div2k_realesrgan_hat_trainer/div2k_venv/lib/python3.11/site-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(
Disable distributed.
Path already exists. Rename it to /Users/gokhankocmarli/Projects/bitirme_tezi/div2k_realesrgan_hat_trainer/.codebases/HAT/experiments/debug_train_HAT-S_SRx2_from_scratch_archived_20231211_004307
2023-12-11 00:43:07,394 INFO: 
                ____                _       _____  ____
               / __ ) ____ _ _____ (_)_____/ ___/ / __ \
              / __  |/ __ `// ___// // ___/\__ \ / /_/ /
             / /_/ // /_/ /(__  )/ // /__ ___/ // _, _/
            /_____/ \__,_//____//_/ \___//____//_/ |_|
     ______                   __  